In [22]:
import pandas as pd 
import numpy as np 

In [23]:
# getting data 
red_df = pd.read_csv('wine+quality/winequality-red.csv')
white_df = pd.read_csv('wine+quality/winequality-white.csv')

In [24]:
X = white_df[["fixed acidity","volatile acidity","citric acid","residual sugar","chlorides","free sulfur dioxide","total sulfur dioxide","density","pH","sulphates","alcohol"]]
y = white_df['quality']

train_index = int(len(X) * .8) # 80% train, 20% test 

X_train = X[:train_index].values
y_train = y[:train_index].values
X_test = X[train_index:].values
y_test = y[train_index:].values
print(f'Size of training data: {len(X_train)}')
print(f'Size of testing data: {len(X_test)}')

Size of training data: 3918
Size of testing data: 980


In [25]:
# predicting y-pred (label), with bias set to zero 
def predict(xi, theta, bias=0):
    label = xi.dot(theta) + bias
    return label

# peforming mse 
def accuracy(theta):
    result = 0
    for i in range(len(X_test)):
        y_pred = (predict(X_test[i], theta))
        result += (y_test[i] - y_pred) ** 2
    return result / len(X_test)

def gaussian_mech_vec(vec, sensitivity, epsilon, delta):
    return [v + np.random.normal(loc=0, scale=sensitivity * np.sqrt(2*np.log(1.25/delta)) / epsilon)
            for v in vec]

def L2_clip(v, b):
    norm = np.linalg.norm(v, ord=2)
    
    if norm > b:
        return b * (v / norm)
    else:
        return v

In [26]:
# This is the gradient of the linear loss
def gradient(theta, xi, yi):
    y_pred = xi.dot(theta) # take current theta and get dot product of x, that is current prediction
    return -2 * (xi * (yi - y_pred))

def noisy_gradient_descent(iterations, epsilon, delta):
    lr = 0.000005 # optimal LR 
    theta = np.zeros(X_train.shape[1])
    epsilon_i = epsilon / iterations
    delta_i = delta / iterations
    for _ in range(iterations):
        # Goal
        # 1. compute one gradient per example in X_train
        all_grads = [gradient(theta, X_train[i], y_train[i]) for i in range(len(X_train))]
        
        # 2. Call L2_clip on each gradient
        b = 5
        clipped_grads = [L2_clip(g, b) for g in all_grads]
        
        # 3. Take the sum of the clipped gradients and add noise
        grad_sum = np.sum(clipped_grads, axis=0)

        # L2 sens is the clipping number
        noisy_grad_sum = gaussian_mech_vec(grad_sum, sensitivity=b, epsilon=epsilon_i, delta=delta_i)
        
        # Danger: reveals the size of the training data - breaks DP
        # Probably fine to reveal, can add noise to the length to make it DP
        noisy_grad = np.array(noisy_grad_sum) / len(X_train)
        theta = theta - lr * noisy_grad
        
    return theta

In [38]:
total_acc = 0
for _ in range(10):
    theta = noisy_gradient_descent(1800, 1.0, 1e-5)
    acc = accuracy(theta)
    total_acc += acc


print(f'Average accuracy for 10 runs, mse: {total_acc/10}')

Average accuracy for 10 runs, mse: 4.153564548955102


In [36]:
accuracy(theta)

np.float64(4.043023345281993)